### 第6週、2日目へようこそ！

より多くのMCPサーバーを試してみましょう

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio

from IPython.display import Markdown, display

load_dotenv(override=True)

True

### 最初のタイプのMCPサーバー: すべてローカルで実行

非常に興味深い例をご紹介します。それは、ナレッジグラフベースのメモリです。

これは、エンティティ、それらに関する観察情報、そしてそれらの関係性を永続的に記憶するものです。

https://github.com/modelcontextprotocol/servers/tree/main/src/memory

- mcp-memory-libsql は、MCPに対応した高性能な永続メモリシステムであり、libSQL（SQLiteのフォーク）をバックエンドとして活用している。
- このサーバーは、AIエージェントや知識グラフ・アプリケーション向けに、ベクトル検索、セマンティック検索、関係管理などの機能を提供する。

In [2]:
# 以下のように実行するらしい。
params = {"command": "npx", "args": ["-y", "mcp-memory-libsql"], "env": {"LIBSQL_URL": "file:./memory/ed.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

In [3]:
# エンティティ・ツールは、会話に関する情報を保存および呼び出すための永続的なメモリとして使用します。"
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."

# Ed です。法学修士号（LLM）を取得しているエンジニアです。AIエージェントに関するコースを担当しており、その中でも素晴らしいMCPプロトコルについて教えています。
# MCPは、エージェントをツール、リソース、プロンプト・テンプレートに接続するためのプロトコルで、AIエージェントと様々な機能の統合を容易にします。
request = "My name's Ed. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."

# エージェントが使用するLLMのモデル
model = "gpt-4.1-mini"

# MCPサーバーを使う処理は async with MCPServerStdio の中で行う必要がある。

In [4]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Nice to meet you, Ed! It sounds like your course on AI Agents and the MCP protocol is very interesting. If you'd like, I can help you organize your course content or provide information related to AI agents, protocols, or anything else you might need for your teaching. Just let me know!

In [6]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        # 私の名前は Ed です。私について何を知っていますか？
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

# ↓、[5]ではツールも使わず知らんと言われてしまったが、2回目の[6]で以下のような回答を得た。

You are Ed. I know that you are an LLM engineer and that you are teaching a course about AI Agents, including the MCP protocol. Is there anything else you'd like me to know or remember about you?

### トレースを確認してください：

https://platform.openai.com/traces

### 2つ目のタイプのMCPサーバー: ローカルで実行され、Webサービスを呼び出す

### Brave Search - 申し訳ありませんが、別のAPIキーが必要になります！ただし、これも無料です。

https://brave.com/search/api/

アカウントを設定し .env に `BRAVE_API_KEY` のキーを入れる。

In [7]:
# 以下のように実行するらしい。
load_dotenv(override=True)
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [8]:
# ウェブで情報を検索し、要点を簡単にまとめることができます。
instructions = "You are able to search the web for information and briefly summarize the takeaways."

# Amazonの株価に関する最新ニュースを調査し、その見通しを簡単にまとめてください。現在の日付は {datetime.now().strftime('%Y-%m-%d')} です。
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"

# エージェントが使用するLLMのモデル
model = "gpt-4o-mini"

# MCPサーバーを使う処理は async with MCPServerStdio の中で行う必要がある。
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

As of September 14, 2025, Amazon's stock (AMZN) has shown a positive trend, recovering after a significant drop in August. Here are the key highlights regarding its current outlook:

1. **Recent Performance**: AMZN shares increased by 4.71% over the past five trading days, indicating a rebound from a previous decline where the stock fell nearly 10% in a single day.

2. **Current Stock Price**: The latest closing price for Amazon is $228.15, with its all-time high for 2025 being $242.06.

3. **Future Predictions**: Analysts estimate that by the end of 2030, Amazon's stock could reach approximately $524.67 per share, projecting a potential upside of about 125%, driven by ongoing revenue growth despite challenges in its AWS segment due to increased competition.

4. **Upcoming Earnings Report**: Amazon is scheduled to release its next earnings report on October 23, 2025, which may further impact its stock performance.

Overall, while Amazon's stock price has been volatile, the long-term outlook appears optimistic according to analysts, emphasizing growth potential in revenue despite market challenges.

### いつものように、トレースをチェックしてください。

https://platform.openai.com/traces

## そして今、3番目のタイプ: リモートで実行されます

「リモートMCPサーバー」別名「ホストされたMCPサーバー」「マネージドMCPサーバー」を見つけるのは、実際には、非常に困難です。

これはMCPサーバーの利用や共有の一般的なモデルではなく、リモートMCPサーバーを見つけるための標準的な方法もありません。

AnthropicはいくつかのリモートMCPサーバーをリストしていますが、これらはビジネスユーザー向けの有料アプリケーション向けです。

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlareには、独自のリモートMCPサーバーを作成および展開するためのツールがありますが、これは一般的な慣行ではないようです。

https://developers.cloudflare.com/agents/guides/remote-mcp-server/

# 結局、2番目のタイプに戻る: Polygon.io MCP Server 

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">必ずお読みください!! -</h2>
            <span style="color:#ff7800;">
                この金融市場データサービスには無料プランと有料プランがあり、お客様のご要望に応じてどちらかをご利用いただけます。
            </span>
        </td>
    </tr>
</table>

## 新しいセクション: Polygon.ioの紹介

Polygon.io は非常に人気のある金融データプロバイダーです。

無料プランと有給計画があります。また、MCPサーバーもあります！

まず、価格を見るなど、優れたウェブサイトで Polygon.io を読んでください。

https://polygon.io

### Polygon.io パート1: Polygon.io 無料サービス（もちろん、有料は完全にオプションになります！）

1. Polygon.io（右上）にサインアップ
2. サインインしたら、左側のナビゲーションで「キー」を選択
3. 青い「新しいキー」ボタンを押す
4. キー名の値をコピーする
5. `.env`ファイルに行を追加: `POLYGON_API_KEY=xxxx`

In [9]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("polygon_api_keyは設定されていません")

In [10]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=234.07, high=234.51, low=229.02, open=229.22, timestamp=1757707200000, volume=55824216.0, vwap=233.1344)

### 終値価格をキャッシュするPythonモジュールにラップされています

このAPIを使って株価を調べるPythonモジュール `market.py` を作成しました。

しかし、無料APIはレート制限がかなり厳しいので、少し工夫してみました。

株価を検索すると、この関数は終値株式市場全体を取得し、データベースにキャッシュします。

In [11]:
from market import get_share_price
get_share_price("AAPL")

234.07

In [12]:
# レート制限の心配はありません!
for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

234.07

### そして、私はこれをMCPサーバーにしました

accounts.pyで行ったように。 `market_server.py`を参照してください

In [13]:
# 自作のMCPサーバの実行
params = {"command": "uv", "args": ["run", "market_server.py"]}

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='lookup_share_price', description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, annotations=None)]

### 試してみましょう！

`gpt-4.1-mini`がAppleのシンボルがAAPLであることを理解できるほど賢いことを願います。

In [14]:
# 株式市場に関する質問に答えます。
instructions = "You answer questions about the stock market."

# Appleの株価はいくらですか？
request = "What's the share price of Apple?"

# エージェントが使用するLLMのモデル（40じゃなくて4.1）
model = "gpt-4.1-mini"

# MCPサーバーを使う処理は async with MCPServerStdio の中で行う必要がある。
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $234.07.

## Polygon.io パート2: 有料プラン - 完全にオプション！
興味がある場合は、毎月の計画を購読して、最新の市場データと無制限のAPI呼び出しを取得できます。

これを希望する場合は、Polygon.ioがリリースした完全なMCPサーバーを使用して、すべての機能を活用することも理にかなっています。

```python
params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools
```

### うわー、それはたくさんのツールです！

それらを試してみましょう - うまくいけば、膨大な数のツールがGPT-4O-MINIを圧倒しません！

毎月29ドルのプランでは、APIの一部にアクセスできないため、どのAPIを呼び出すことができるかを指定する必要がありました。

あなたがより大きな計画に跳ね返った場合は、私の追加の制約を削除してください。

```python
# ※ request で 使用ツールが明言されている。

# 株式市場に関する質問に答えます。
instructions = "You answer questions about the stock market."

# Appleの株価はいくらですか？ get_snapshot_ticker ツールを使用して最新の価格を取得してください。
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
# エージェントが使用するLLMのモデル（40じゃなくて4.1）
model = "gpt-4.1-mini"

# MCPサーバーを使う処理は async with MCPServerStdio の中で行う必要がある。
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))
```

## .env ファイルの設定

有料プランをご利用になる場合は、.env ファイルに以下の記述を追加してください。

`POLYGON_PLAN=paid`

リアルタイム API をご利用になる場合は、以下の記述を追加してください。

`POLYGON_PLAN=realtime`

```python
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("有料のPolygonプランに加入することを選択したため、コードは15分遅れで価格を確認します。")
elif is_realtime_polygon:
    print("すごいですね。リアルタイムのPolygonプランに加入することを選択したので、コードはリアルタイムの価格を参照します。")
else:
    print(".envファイルによると、無料のPolygonプランに加入することを選択したため、コードはEOD価格を参照します。")
```

## そして、それは今日のためです！

「金融データセット」MCPサーバーを使用するこのラボの一部を削除しました。これは劣っているためです。APIが少なく、より高価です。

このようにして、同じプロバイダーを無料および有料APIで使用することができます。

ただし、コードを表示したい場合は、以前のバージョンについてはGit履歴を確認してください。

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">演習</h2>
            <span style="color:#ff7800;">
                3 つのアプローチすべてを使用して、MCP サーバー マーケットプレイスを調査し、独自のマーケットプレイスを統合します。
            </span>
        </td>
    </tr>
</table>